# Importing essential library

In [1]:
# essentials
import shutil
import os

# multiprocessing
import multiprocessing 

# dataset
from tqdm import tqdm 
import pandas as pd
import numpy as np 

# webscraping
import requests
from bs4 import BeautifulSoup
import time

# webdriver 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Installing the webdriver

In [3]:
browser = webdriver.Chrome(ChromeDriverManager().install())

link = 'https://www.imdb.com/search/name/?match_all=true&start=0&ref_=rlm'
browser.get(link)
soup = BeautifulSoup(browser.page_source,'html.parser')

/tmp/ipykernel_3289/4103546689.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


### Defining a profile scraper

In [4]:
def profile_scraper(start,end):
    celeb_data = []
    # initial link 
    link = 'https://www.imdb.com/search/name/?match_all=true&start=0&ref_=rlm'
    for pages in tqdm(range(start,end)):   

        browser.get(link)

        soup = BeautifulSoup(browser.page_source,'html.parser')

        for celeb in soup.find_all('div',class_ = 'lister-item mode-detail'):
            try:
                img_link     = celeb.find('a').find('img').get('src')
            except:
                img_link
            try:
                celeb_id     = celeb.find('div',class_ = 'lister-item-content').find('a').get('href')[6:]
            except:
                celeb_id
            try:
                profile_link = 'https://www.imdb.com' + celeb.find('div',class_ = 'lister-item-content').find('a').get('href')
            except:
                profile_link
            try:
                celeb_name   = celeb.find('div',class_ = 'lister-item-content').find('a').text.strip()
            except:
                celeb_name
            try:
                prof         = BeautifulSoup(str(celeb.find('div',class_ = 'lister-item-content').find('p')).split('|')[0],'html.parser')
                fame         = BeautifulSoup(str(celeb.find('div',class_ = 'lister-item-content').find('p')).split('|')[1],'html.parser')
                profession   = prof.text.strip()
                famous_for   = fame.text.strip()
            except:
                profession = 'Not Available'
                famous_for = 'Not Available'

            celeb_data.append([celeb_id,celeb_name,profession,famous_for,img_link,profile_link])

        link = 'https://www.imdb.com'+soup.find('div',class_ = 'desc').find('a',class_ = 'lister-page-next next-page').get('href')
    celeb = pd.DataFrame(celeb_data,columns = ['celeb_id','celeb_name','profession','famous_for','img_link','profile_link'])
    celeb.head()

In [ ]:
p1 = multiprocessing.Process(target = profile_scraper,args =(0,101))
p2 = multiprocessing.Process(target = profile_scraper,args =(101,201))
p3 = multiprocessing.Process(target = profile_scraper,args =(201,301))
p4 = multiprocessing.Process(target = profile_scraper,args =(301,401))
p5 = multiprocessing.Process(target = profile_scraper,args =(401,501))
p6 = multiprocessing.Process(target = profile_scraper,args =(501,601))

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()
p6.start()

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()
p6.join()


In [90]:
celeb.to_csv('celebs.csv',index = False)

In [92]:
celeb.shape

(25047, 6)

### Downloading the images | Multiprocessing 

In [13]:
df = pd.read_csv('celeb(1).csv')
df.isnull().sum()

celeb_id        0
celeb_name      0
profession      0
famous_for      0
img_link        0
profile_link    0
dtype: int64

In [65]:
! mkdir celeb_images

In [88]:
def download_img(data,start,end):
    for i in tqdm(df.values[start:end]):
        r = requests.get(i[4], stream = True)
        name = i[1].replace(' ','_')                   # saving the name of th celebs
        path = f'celeb_images/{name}.png' 
        if r.status_code == 200:                       # check if the response is 200
            with open(path, 'wb') as f:                # opening a file as web
                r.raw.decode_content = True            # decoding the image 
                shutil.copyfileobj(r.raw, f)           # downloading the image to path  

In [93]:
p1 = multiprocessing.Process(target = download_img,args =(df,0,1001))
p2 = multiprocessing.Process(target = download_img,args =(df,1001,2001))
p3 = multiprocessing.Process(target = download_img,args =(df,2001,3001))
p4 = multiprocessing.Process(target = download_img,args =(df,3001,4001))
p5 = multiprocessing.Process(target = download_img,args =(df,4001,4999))

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()



100%|█████████████████████████████████████████| 998/998 [09:58<00:00,  1.67it/s]
